# For users who don't have a GPU!

In [ ]:
import os

# Set up the environment.
%cd /content
if(not os.path.exists("manga-image-translator")):
  !git clone --depth 1 "https://github.com/zyddnys/manga-image-translator"
  %cd manga-image-translator/
  !python -m pip install -r requirements.txt
  !python -m pip install --force-reinstall cython==0.29.36
  !python -m pip install --no-build-isolation git+https://github.com/lucasb-eyer/pydensecrf.git

  # To expose service outside
  !npm i localtunnel -g

In [ ]:
%cd /content/manga-image-translator

import psutil, subprocess, os

# Get a list of all processes that are listening on specified ports
ports = [5003, 5004]
for proc in psutil.process_iter():
    try:
        for conn in proc.connections():
            if conn.laddr.port in ports:
                print(f"Killing process {proc.pid}...")
                proc.kill()
    except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
        pass

proc = subprocess.Popen(['python', '-m', "manga_translator", "--verbose", "--mode", "web", "--use-cuda"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
ltngproc = subprocess.Popen(['lt', '--port', "5003"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

print("Runner IP:", subprocess.run(["curl", "ipv4.icanhazip.com"], stdout=subprocess.PIPE).stdout.decode())

def read(x):
  for line in x.stdout:
    print(line.decode("utf-8"))

from threading import Thread
threads = [
    Thread(target=read, args=(proc,)),
    Thread(target=read, args=(ltngproc,))
]
for t in threads:
  t.daemon = True
  t.start()

for t in threads:
  t.join()